In [48]:
# Librairies de base
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
sns.set_style("darkgrid")

# Fonctions de la librairie sklearn pour l'apprentissage statistiques (Machine Learning)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

# Librairies pour le traitement du langage

import nltk
from nltk.stem import WordNetLemmatizer
from langdetect import detect
 
import re
import string
from nltk.corpus import words

# Librairies d'apprentissage profond
from gensim.test.utils import datapath
from gensim import utils
import gensim
import gensim.downloader as api
import tensorflow as tf
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mehdyhkn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [23]:
import tqdm
from nlp_utils import *

In [6]:
total = pd.read_csv("Table_sent1.csv")


In [7]:

total.head()

,Unnamed: 0,User,Id_listing,comments,sentiment,room_type,price,number_of_reviews,calculated_host_listings_count,price_group,review_scores_rating,categorie_score
0,1,Dallas,218103,Luis' bed and breakfast is just fantastic. Hi...,9,Entire home/apt,110,39,4,Luxe,94,Moyen
1,2,Dallas,218103,Four of us had the pleasure of staying in two ...,9,Entire home/apt,110,39,4,Luxe,94,Moyen
2,3,Dallas,218103,What a fantastic host and a brilliant place!!!...,9,Entire home/apt,110,39,4,Luxe,94,Moyen
3,4,Dallas,218103,my friends and i ( 5ppl in total) had pleasant...,9,Entire home/apt,110,39,4,Luxe,94,Moyen
4,5,Dallas,218103,Luis was super great! Fantastic host who was h...,9,Entire home/apt,110,39,4,Luxe,94,Moyen


In [8]:
total.shape

(992788, 12)

In [9]:
total["commentaires"] = [str(x) for x in total.comments]

In [10]:
total.head()

,Unnamed: 0,User,Id_listing,comments,sentiment,room_type,price,number_of_reviews,calculated_host_listings_count,price_group,review_scores_rating,categorie_score,commentaires
0,1,Dallas,218103,Luis' bed and breakfast is just fantastic. Hi...,9,Entire home/apt,110,39,4,Luxe,94,Moyen,Luis' bed and breakfast is just fantastic. Hi...
1,2,Dallas,218103,Four of us had the pleasure of staying in two ...,9,Entire home/apt,110,39,4,Luxe,94,Moyen,Four of us had the pleasure of staying in two ...
2,3,Dallas,218103,What a fantastic host and a brilliant place!!!...,9,Entire home/apt,110,39,4,Luxe,94,Moyen,What a fantastic host and a brilliant place!!!...
3,4,Dallas,218103,my friends and i ( 5ppl in total) had pleasant...,9,Entire home/apt,110,39,4,Luxe,94,Moyen,my friends and i ( 5ppl in total) had pleasant...
4,5,Dallas,218103,Luis was super great! Fantastic host who was h...,9,Entire home/apt,110,39,4,Luxe,94,Moyen,Luis was super great! Fantastic host who was h...


In [11]:
total.shape

(992788, 13)

In [12]:
total["description_lower"] = [x.lower() for x in total.commentaires] #lower 
total["description_lower"] = [''.join([i for i in s if not i.isdigit()]) for s in total["description_lower"]] #enlevage des chiffres

In [13]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub('', s).split()

In [14]:
train = [tokenize(i) for i in total.description_lower]

In [15]:
# Lemmatisation du jeu de données d'apprentissage
description_token = train

lemmatizer = WordNetLemmatizer()
description = []

for i in train:
    lemmatized = []
    for word, tag in nltk.pos_tag(i):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag : 
            lemmatized.append(word)
        else : 
            lemmatized.append(lemmatizer.lemmatize(word,wntag))
    description.append(' '.join(lemmatized))

total["description_lemmatized"] = description

# Visualisation de la lemmatisation
total.head()

,Unnamed: 0,User,Id_listing,comments,sentiment,room_type,price,number_of_reviews,calculated_host_listings_count,price_group,review_scores_rating,categorie_score,commentaires,description_lower,description_lemmatized
0,1,Dallas,218103,Luis' bed and breakfast is just fantastic. Hi...,9,Entire home/apt,110,39,4,Luxe,94,Moyen,Luis' bed and breakfast is just fantastic. Hi...,luis' bed and breakfast is just fantastic. hi...,luis bed and breakfast be just fantastic his c...
1,2,Dallas,218103,Four of us had the pleasure of staying in two ...,9,Entire home/apt,110,39,4,Luxe,94,Moyen,Four of us had the pleasure of staying in two ...,four of us had the pleasure of staying in two ...,four of us have the pleasure of stay in two ro...
2,3,Dallas,218103,What a fantastic host and a brilliant place!!!...,9,Entire home/apt,110,39,4,Luxe,94,Moyen,What a fantastic host and a brilliant place!!!...,what a fantastic host and a brilliant place!!!...,what a fantastic host and a brilliant place we...
3,4,Dallas,218103,my friends and i ( 5ppl in total) had pleasant...,9,Entire home/apt,110,39,4,Luxe,94,Moyen,my friends and i ( 5ppl in total) had pleasant...,my friends and i ( ppl in total) had pleasant ...,my friend and i ppl in total have pleasant sta...
4,5,Dallas,218103,Luis was super great! Fantastic host who was h...,9,Entire home/apt,110,39,4,Luxe,94,Moyen,Luis was super great! Fantastic host who was h...,luis was super great! fantastic host who was h...,luis be super great fantastic host who be help...


In [16]:
#decoupage train/test
Xtrain,Xtest,Ytrain,Ytest = train_test_split(total["description_lemmatized"],total.categorie_score.values,test_size = 0.2, random_state=42)
print(Xtrain)

929892    great value for a night in bordeaux so close t...
979310    what a wonderful stay at brussaignes place in ...
939227    laurent and wilson truly care about their rent...
73859     great apartment and host you could not ask for...
326073    love the place ac be great as it be so hot in ...
147610    xavier apartment be in the best location and o...
852939    stay in the city a few time but this be the pl...
886557    it be a pleasant stay in gaetan et gregs apart...
794637    the apartment be in a great location within wa...
308878    we have a great stay the apartment be very wel...
3610      this small apartment be in a great location in...
414230    we have be to airbnbs and our bordeaux visit b...
412514    stephane be so welcoming meeting us at his apa...
941584    fred be very helpful with transport from the a...
44435     bordeaux be such a charming city we love it fr...
310087    xavier place be in a great location and perfec...
935099          nice place good location

In [3]:
total.head()

NameError: name 'total' is not defined

In [17]:
#transformer = TfidfVectorizer().fit(total["description_lemmatized"].values)
transformer = TfidfVectorizer(stop_words="english").fit(total["description_lemmatized"].values)

In [18]:
# On convertit notre Xtrain et Xtest avec la transformation TF-IDF
Xtrain_baseline = transformer.transform(Xtrain)
Xtest_baseline = transformer.transform(Xtest)

In [19]:
model = LogisticRegression() #n_jobs permet de choisir le nombre de coeur du processeur utilisés 
# On fit notre modèle régression logistique avec par défaut une pénalité ridge
model.fit(Xtrain_baseline, Ytrain)



/Users/mehdyhkn/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/mehdyhkn/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
# On effectue une prédiction
predictions = model.predict(Xtest_baseline)
# On recupère les probas
proba = model.predict_proba(Xtest_baseline)

In [21]:
sum(Ytest == predictions)/len(Ytest)

0.8179020739532026

In [52]:
print(classification_report(Ytest,predictions))

              precision    recall  f1-score   support

   Excellent       0.83      0.82      0.83     68108
       Haute       0.84      0.73      0.78     56640
       Moyen       0.79      0.88      0.84     73808
    Médiocre       0.00      0.00      0.00         2

    accuracy                           0.82    198558
   macro avg       0.62      0.61      0.61    198558
weighted avg       0.82      0.82      0.82    198558



In [60]:
Ytest[1:5]


array(['Haute', 'Haute', 'Excellent', 'Excellent'], dtype=object)

In [55]:
predictions[1:5]

array(['Haute', 'Haute', 'Excellent', 'Moyen'], dtype=object)

In [56]:
conf= confusion_matrix(Ytest[1:5], predictions[1:5])
#cf = pd.DataFrame(conf, columns=['prédit Haute', 'prédit Moyen', 'prédit Médiocre'])
#cf.index = ['vrai ham', 'vrai spam']
conf

array([[1, 0, 1],
       [0, 2, 0],
       [0, 0, 0]])

In [45]:
sum(predictions == "Haute")


49009

In [100]:
#decoupage train/test
Xtrain_,Xtest_,Ytrain_,Ytest_ = train_test_split(total["description_lemmatized"],total.review_scores_rating.values,test_size = 0.2, random_state=42)
print(Xtrain_)

929892    great value for a night in bordeaux so close t...
979310    what a wonderful stay at brussaignes place in ...
939227    laurent and wilson truly care about their rent...
73859     great apartment and host you could not ask for...
326073    love the place ac be great as it be so hot in ...
147610    xavier apartment be in the best location and o...
852939    stay in the city a few time but this be the pl...
886557    it be a pleasant stay in gaetan et gregs apart...
794637    the apartment be in a great location within wa...
308878    we have a great stay the apartment be very wel...
3610      this small apartment be in a great location in...
414230    we have be to airbnbs and our bordeaux visit b...
412514    stephane be so welcoming meeting us at his apa...
941584    fred be very helpful with transport from the a...
44435     bordeaux be such a charming city we love it fr...
310087    xavier place be in a great location and perfec...
935099          nice place good location

In [102]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [104]:
transformer_ = TfidfVectorizer(stop_words="english").fit(total["description_lemmatized"].values)

In [107]:
# On convertit notre Xtrain et Xtest avec la transformation TF-IDF
Xtrain_1 = transformer_.transform(Xtrain_)
Xtest_1 = transformer_.transform(Xtest_)

In [110]:
lmodellineaire = LinearRegression()
lmodellineaire.fit(Xtrain_1, Ytrain_)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [111]:
# Evaluation du training set
from sklearn.metrics import r2_score
y_train_predict = lmodellineaire.predict(Xtrain_1)
rmse = (np.sqrt(mean_squared_error(Ytrain_, y_train_predict)))
r2 = r2_score(Ytrain_, y_train_predict)
 
print('La performance du modèle sur la base dapprentissage')
print('--------------------------------------')
print('Lerreur quadratique moyenne est {}'.format(rmse))
print('le score R2 est {}'.format(r2))
print('\n')
 
# model evaluation for testing set
y_test_predict = lmodellineaire.predict(Xtest_1)
rmse = (np.sqrt(mean_squared_error(Ytest_, y_test_predict)))
r2 = r2_score(Ytest_, y_test_predict)
 
print('La performance du modèle sur la base de test')
print('--------------------------------------')
print('Lerreur quadratique moyenne est {}'.format(rmse))
print('le score R2 est {}'.format(r2))

La performance du modèle sur la base dapprentissage
--------------------------------------
Lerreur quadratique moyenne est 1.9670657643739022
le score R2 est 0.6358493633833624


La performance du modèle sur la base de test
--------------------------------------
Lerreur quadratique moyenne est 1.9991469536015554
le score R2 est 0.6229673256216983
